### Model-free Methods
- Previously we explored dynamic programming for finding the optimal policy for our frozenlake game
    - In that frozenlake game, we knew our rewards and transition probability functions
        - In the deterministic environment, whatever action we took would have a 100% probability of landing in a following state we knew
        - In the stochastic environment, whatever action we took would have a 1/3 probability each of landing in 3 different states
            - In the known [transition probability function example](https://www.deeplearningwizard.com/deep_learning/deep_reinforcement_learning_pytorch/dynamic_programming_frozenlake/#transition-probability-function), if we took left (action = 0) given we are in state 10, we would end up in either 9 (left of state 10), 6 (above of state 10) or 14 (below of state 10) each with 1/3 probability
    - Because we know our transition probability function and rewards, it essentially becomes a planning problem solvable by dynamic programming
- But in the cases where we do not have transition probability function, we need model-free methods (essentially trial-and-error methods)
    - We will be exploring the Blackjack environment
    - In this case, when we take or pass a card, because we do not know what card

### Exploring Blackjack Environment

#### Observational and Action Space
- Action space
    - Stick: don't take a card (pass)
        - This must be the final action, otherwise you can hit (take a card) multiple times before this action
    - Hit: take a card
- Observational space
    - player's current sum of all cards
        - $\{0, 1, \ldots, 31\}$
        - where 1 = Ace, 2-10 = Number cards, Jack/Queen/King = 10
    - dealer's face-up card
        - $\{1, \ldots, 10\}$
        - where 1 = Ace, 2-10 = Number cards, Jack/Queen/King = 10
    - player's usable ace
        - $\{0, 1\}$
        - where 0 = no, 1 = yes
        

   

In [2]:
# Import gym
import gym

# Make the environment
env = gym.make('Blackjack-v0')

# Tuple of (player sum of cards, dealer's face up card, player's usable ace)
print(env.observation_space)

# STICK (don't take a card) or HIT (take a card)
print(env.action_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))
Discrete(2)


#### 2 Game Trial

In [46]:
# Set environment seed for replicability
env.seed(1337)
env.action_space.seed(1337)

# Run through 2 games
for game_idx in range(2):
    # Reset environment
    state = env.reset()
    print('='*50)
    print('Game number {}'.format(game_idx + 1))
    
    # Game 1: Player sum of cards = 12, Dealer's open card = 4, Player no Ace
    # Game 2: Player sum of cards = 13, Dealer's open card = 8, Player no Ace
    print('Initial state {}'.format(state))

    while True:
        # Randomly sample action from the action space, still will randomly return 0 or 1
        # action = 1 means take a card, action = 2 means pass
        action = env.action_space.sample()
        if action == 1:
            print('Hit, action = {}'.format(action))
        else:
            print('Pass, action = {}'.format(action))
        
        state, reward, done, _ = env.step(action)
        
        if done:
            print('Final state {}'.format(state))
            print('End game, Reward = {}'.format(reward))
            
            if reward > 0:
                print('Won game.')
            else:
                print('Lost game.')
            break

Game number 1
Initial state (12, 4, False)
Hit, action = 1
Final state (22, 4, False)
End game, Reward = -1
Lost game.
Game number 2
Initial state (13, 8, False)
Hit, action = 1
Pass, action = 0
Final state (18, 8, False)
End game, Reward = 1.0
Won game.


### MC methods

#### Sample Policy Function
- Sample a preset policy based on the following configuration
    - If the player's sum of cards > 16
        - STICK probability = 80%
        - HIT probability = 20%
    - If the player's sum of cards <= 16
        - STICK probability = 20%
        - HIT probability = 80%
- Essentially we want the player to be inclined to take a card (HIT) when the sum of cards is not high

In [48]:
import numpy as np

# Set seeds for reproducibility
env.seed(100)
np.random.seed(100)

def sample_policy(env):
    episode = []
    # Reset the environment
    state = env.reset()
    
    # Infinite loop until done
    # only done when player STICK which is to pass to take a card (pass to HIT essentially)
    while True:
        # If the player's sum is more than 16, we STICK with 80% prob and HIT with 20% prob
        if state[0] > 16:
            action_probabilities = [0.8, 0.2]
        # If the player's sum is less than or equal to 16, we HIT with 80% probability and STICK with 20% probability
        else:
            action_probabilities = [0.2, 0.8]
        
        # Get action based on action probabilities
        action = np.random.choice(np.arange(2), p=action_probabilities)
        
        # Pass action to environment
        next_state, reward, done, _ = env.step(action)
        
        # Append the state prior to the action for log
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode


# Loop through 2 games

# First game: 
#     sum of cards <= 16, action sampled is STICK (20% probability)

# Second game:
#     sum of cards <= 16, action sampled is HIT (80% probability)
#     sum of cards still <= 16, action sampled is STICK (20% probability)
for i in range(4):
    print(sample_policy(env))

[((14, 8, False), 1, -1)]
[((11, 4, False), 1, 0), ((14, 4, False), 1, -1)]
[((18, 6, False), 1, 0), ((20, 6, False), 0, 1.0)]
[((14, 5, False), 0, -1.0)]


#### MC Prediction Function
- This function obtains the action-value (q-value) function estimate $Q$

In [53]:
# This function defaultdicts prevents returning of key error when there is no key found when indexing
# instead it will return the type of the argument passed to defaultdict
# if integer function is passed, it will return 0
# if numpy array function is passed, it will return the numpy array with 0's filled
from collections import defaultdict

# This is to enable sys.stdout.flush() for single line clean printing of status
import sys

# 
def mc_prediction_q(env, num_episodes, generate_episode, gamma=1.0):
    # initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            # This prevents you from printing multiple lines. Replaces existing line with new log.
            sys.stdout.flush()
        # generate an episode
        episode = generate_episode(env)
        # obtain the states, actions, and rewards
        states, actions, rewards = zip(*episode)
        # prepare for discounting
        discounts = np.array([gamma**i for i in range(len(rewards)+1)])
        # update the sum of the returns, number of visits, and action-value 
        # function estimates for each state-action pair in the episode
        for i, state in enumerate(states):
            returns_sum[state][actions[i]] += sum(rewards[i:]*discounts[:-(1+i)])
            N[state][actions[i]] += 1.0
            Q[state][actions[i]] = returns_sum[state][actions[i]] / N[state][actions[i]]
    return Q

In [ ]:
# obtain the action-value function
Q = mc_prediction_q(env, 500000, generate_episode_from_limit_stochastic)

# obtain the corresponding state-value function
V_to_plot = dict((k,(k[0]>18)*(np.dot([0.8, 0.2],v)) + (k[0]<=18)*(np.dot([0.2, 0.8],v))) for k, v in Q.items())

In [ ]:
# To use model-based methods we need to have complete knowledge of the environment
#i.e. we need to know Pss’ (please refer to above picture): the transition probability we’ll end up in state St+1=s’ if the agent is in state St=1 and takes action At=a. For example, if a bot chooses to move forward, it might move sideways in case of slippery floor underneath it. In games like Blackjack our action space is limited like we can either choose to “hit” or “stick” but we can end up in any of the many possible states of which you know none of the probabilities! In Blackjack state is determined by your sum, the dealers sum and whether you have a usable ace or not as follows:

Episode 500000/500000.